In [98]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [99]:
spark= SparkSession.builder.getOrCreate()

In [100]:
players=spark.read.format('csv')\
    .option('header','True') \
    .option('inferSchema','True') \
    .load('../data/players.csv')

In [101]:
players.show()

+---------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+
| LastName|FirstName|     Date|Start|Pos|Min|  G|  A| PK|PKA|  S|SoT| YK| RK|Touches|Tackles|Ints|Blocks| xG|npxG|xAG|Passes|PassesA|PrgPas|Carries|PrgCar|      Line|  C|
+---------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+
| Bellerin|   Hector|8/11/2017|    1| WB| 90|  0|  0|  0|  0|  1|  1|  0|  0|     79|      3|   0|     0|0.3| 0.3|0.0|    61|     70|     3|     51|     1|  Defender|  0|
|   Elneny|  Mohamed|8/11/2017|    1| CM| 66|  0|  1|  0|  0|  1|  0|  0|  0|     82|      4|   0|     2|0.0| 0.0|0.1|    65|     72|     4|     57|     0|Midfielder|  0|
|  Holding|      Rob|8/11/2017|    1| CB| 66|  0|  0|  0|  0|  0|  0|  0|  0|     75|      1|   1|     0|0.0| 0.0|0.0|    50|     60|     4|     

In [102]:
players.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C']

In [103]:
players.schema

StructType([StructField('LastName', StringType(), True), StructField('FirstName', StringType(), True), StructField('Date', StringType(), True), StructField('Start', IntegerType(), True), StructField('Pos', StringType(), True), StructField('Min', IntegerType(), True), StructField('G', IntegerType(), True), StructField('A', IntegerType(), True), StructField('PK', IntegerType(), True), StructField('PKA', IntegerType(), True), StructField('S', IntegerType(), True), StructField('SoT', IntegerType(), True), StructField('YK', IntegerType(), True), StructField('RK', IntegerType(), True), StructField('Touches', IntegerType(), True), StructField('Tackles', IntegerType(), True), StructField('Ints', IntegerType(), True), StructField('Blocks', IntegerType(), True), StructField('xG', DoubleType(), True), StructField('npxG', DoubleType(), True), StructField('xAG', DoubleType(), True), StructField('Passes', IntegerType(), True), StructField('PassesA', IntegerType(), True), StructField('PrgPas', Intege

In [104]:
players.createOrReplaceTempView("players")

In [105]:
spark.sql('''
        select concat(firstname," ",lastname) as fullname
        from players
          
          ''').count()

2741

In [106]:
distinct_players=spark.sql('''
        select distinct concat(firstname," ",lastname) as fullname
        from players
          
          ''')

In [107]:
players_dates=spark.sql('''
    select count(distinct Date)
    from players

''')

In [108]:
players_dates.show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 214|
+--------------------+



## Here we make sure that the distinct count date for matches in DimMatches Equal the distinct count Date for DimPlayers 214

In [109]:
distinct_players=distinct_players.withColumn('PlayersID',monotonically_increasing_id())

In [110]:
distinct_players.show()

+-------------------+---------+
|           fullname|PlayersID|
+-------------------+---------+
|   Emile Smith Rowe|        0|
|    Folarin Balogun|        1|
|    Hector Bellerin|        2|
|        Joe Willock|        3|
|     William Saliba|        4|
|       Aaron Ramsey|        5|
|        Bukayo Saka|        6|
|     Kieran Tierney|        7|
|   Shkodran Mustafi|        8|
|    Daniel Ceballos|        9|
|Alexandre Lacazette|       10|
|     Kieran Willian|       11|
|      Gabriel Jesus|       12|
|      Danny Welbeck|       13|
| Gabriel Marquinhos|       14|
|         Pablo Mari|       15|
|          Ben White|       16|
|      Ethan Nwaneri|       17|
|     Sead Kolasinac|       18|
|     Calum Chambers|       19|
+-------------------+---------+
only showing top 20 rows



In [111]:
players=players.withColumn('fullname',concat_ws(' ',col('FirstName'),col('LastName')))

In [112]:
players.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'fullname']

In [113]:
players.select('fullname').show(5)

+-------------------+
|           fullname|
+-------------------+
|    Hector Bellerin|
|     Mohamed Elneny|
|        Rob Holding|
|     Sead Kolasinac|
|Alexandre Lacazette|
+-------------------+
only showing top 5 rows



In [114]:
dimplayer=players.join(distinct_players,on='fullname',how='inner')

In [115]:
dimplayer.columns

['fullname',
 'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'PlayersID']

In [116]:
dimplayer.write.csv('../data/Dimplayers.csv', header=True)

In [117]:
dimplayer.select('PlayersID',
'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA' 
).show()

+---------+---------+---------+---------+-----+---+---+---+---+---+---+
|PlayersID| LastName|FirstName|     Date|Start|Pos|Min|  G|  A| PK|PKA|
+---------+---------+---------+---------+-----+---+---+---+---+---+---+
|        2| Bellerin|   Hector|8/11/2017|    1| WB| 90|  0|  0|  0|  0|
|       49|   Elneny|  Mohamed|8/11/2017|    1| CM| 66|  0|  1|  0|  0|
|       26|  Holding|      Rob|8/11/2017|    1| CB| 66|  0|  0|  0|  0|
|       18|Kolasinac|     Sead|8/11/2017|    1| CB| 90|  0|  1|  0|  0|
|       10|Lacazette|Alexandre|8/11/2017|    1| FW| 90|  1|  0|  0|  0|
|       21|  Monreal|    Nacho|8/11/2017|    1| CB| 90|  0|  0|  0|  0|
|       53|     Ozil|    Mesut|8/11/2017|    1| AM| 90|  0|  0|  0|  0|
|       13|  Welbeck|    Danny|8/11/2017|    1| AM| 74|  1|  0|  0|  0|
|       44|    Xhaka|   Granit|8/11/2017|    1| CM| 90|  0|  2|  0|  0|
|        5|   Ramsey|    Aaron|8/11/2017|    0| DM| 24|  1|  0|  0|  0|
|        2| Bellerin|   Hector|8/19/2017|    1| WB| 90|  0|  0| 

### Creating the FactPlayers

In [118]:
dimmatch=spark.read.format('csv')\
    .option('header','True')\
    .option('inferSchema','True')\
    .load('../data/DimMatches/DimMatches.csv')

In [119]:
dimmatch.columns

['Season',
 'Tour',
 'Date',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee',
 'MatchID']

In [120]:
dimplayers=spark.read.format('csv')\
    .option('header','True')\
    .option('inferSchema','True')\
    .load("../data/Dimplayers/DimPlayers.csv")

In [121]:
dimplayers.columns

['fullname',
 'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'PlayersID']

In [122]:
factplayers=dimmatch.join(dimplayers,on='Date',how='left')

In [123]:
factplayers.show()

+----------+-------+----+-------------------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+--------+--------+---------+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+----+----+------+-------+------+-------+------+----+----+---------+
|      Date| Season|Tour|               Time|      Opponent|HoAw|ArsenalScore|OpponentScore|          Stadium|Attendance|        Coach|        Referee|MatchID|fullname|LastName|FirstName|Start| Pos| Min|   G|   A|  PK| PKA|   S| SoT|  YK|  RK|Touches|Tackles|Ints|Blocks|  xG|npxG| xAG|Passes|PassesA|PrgPas|Carries|PrgCar|Line|   C|PlayersID|
+----------+-------+----+-------------------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+--------+--------+---------+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+----+----+------+-------+------+----

In [124]:
factplayers.columns

['Date',
 'Season',
 'Tour',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee',
 'MatchID',
 'fullname',
 'LastName',
 'FirstName',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'PlayersID']

In [125]:
factplayers=factplayers.drop(
     'fullname',
 'LastName',
 'FirstName',
 'Line',
 'Stadium',
 'Coach',
 'Referee',
 'Opponent',
 'HoAw',
 'Season',
 'Tour',
 'Time',
 'C'
)

In [126]:
factplayers.columns

['Date',
 'ArsenalScore',
 'OpponentScore',
 'Attendance',
 'MatchID',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'PlayersID']

In [127]:
factplayers.write.csv('../data/Factplayers', header=True)

### Transforming Players Data & Creating the DimPlayers and Fact Players

In [128]:
dimplayer=spark.read.csv('../data/Dimplayers/DimPlayers.csv',header=True)

In [129]:
dimplayer.show()

+-------------------+---------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+---------+
|           fullname| LastName|FirstName|     Date|Start|Pos|Min|  G|  A| PK|PKA|  S|SoT| YK| RK|Touches|Tackles|Ints|Blocks| xG|npxG|xAG|Passes|PassesA|PrgPas|Carries|PrgCar|      Line|  C|PlayersID|
+-------------------+---------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+---------+
|    Hector Bellerin| Bellerin|   Hector|8/11/2017|    1| WB| 90|  0|  0|  0|  0|  1|  1|  0|  0|     79|      3|   0|     0|0.3| 0.3|0.0|    61|     70|     3|     51|     1|  Defender|  0|        2|
|     Mohamed Elneny|   Elneny|  Mohamed|8/11/2017|    1| CM| 66|  0|  1|  0|  0|  1|  0|  0|  0|     82|      4|   0|     2|0.0| 0.0|0.1|    65|     72|     4|     57|     0|Midfielder|  0|      

In [130]:
dimplayer.columns

['fullname',
 'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'PlayersID']

In [131]:
dimplayer=dimplayer.drop('Date',
 'Start',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C','FormattedDate', 'Pos')

In [132]:
dimplayer.columns

['fullname', 'LastName', 'FirstName', 'PlayersID']

In [139]:
dimplayer.show()

+-------------------+---------+---------+---------+
|           fullname| LastName|FirstName|PlayersID|
+-------------------+---------+---------+---------+
|    Hector Bellerin| Bellerin|   Hector|        2|
|     Mohamed Elneny|   Elneny|  Mohamed|       49|
|        Rob Holding|  Holding|      Rob|       26|
|     Sead Kolasinac|Kolasinac|     Sead|       18|
|Alexandre Lacazette|Lacazette|Alexandre|       10|
|      Nacho Monreal|  Monreal|    Nacho|       21|
|         Mesut Ozil|     Ozil|    Mesut|       53|
|      Danny Welbeck|  Welbeck|    Danny|       13|
|       Granit Xhaka|    Xhaka|   Granit|       44|
|       Aaron Ramsey|   Ramsey|    Aaron|        5|
|    Hector Bellerin| Bellerin|   Hector|        2|
|     Sead Kolasinac|Kolasinac|     Sead|       18|
|Alexandre Lacazette|Lacazette|Alexandre|       10|
|      Nacho Monreal|  Monreal|    Nacho|       21|
|   Shkodran Mustafi|  Mustafi| Shkodran|        8|
|         Mesut Ozil|     Ozil|    Mesut|       53|
|       Aaro

#### Loading the DimPlayers for DWH Schema in ArsenalFC Database in PostgreSql

In [138]:
dimplayer.schema

StructType([StructField('fullname', StringType(), True), StructField('LastName', StringType(), True), StructField('FirstName', StringType(), True), StructField('PlayersID', StringType(), True)])

In [143]:
dimplayer.write.format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/mydatabase") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "dwh.DimArsenalPlayers") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .mode("overwrite") \
    .save()

Py4JJavaError: An error occurred while calling o464.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at or